## 실습 문제: N-Queens 문제 정의 및 목적 함수 구현

**설명**:
Local Search를 적용하기 위해 N-Queens 문제를 `Problem` 클래스를 상속받아 정의합니다.
상태(State)는 각 열(Column)에 있는 퀸의 행(Row) 인덱스를 담은 튜플로 표현합니다. 예를 들어 `N=4`일 때 `(1, 3, 0, 2)`는 0번 열의 퀸이 1번 행, 1번 열의 퀸이 3번 행에 있음을 의미합니다.

N-Queens 문제에서 가장 중요한 것은 현재 상태가 얼마나 좋은지를 평가하는 \*\*목적 함수(Objective Function)\*\*입니다. 여기서는 서로 공격하는 퀸의 쌍(Attacking Pairs)의 개수를 구하며, 이 값이 0이 되면 목표 상태입니다.

**요구사항**:

1.  `objective_function`: 현재 상태에서 서로 공격 가능한 퀸의 쌍의 개수를 반환하도록 구현하세요.
      * **수평 공격**: 두 퀸의 행(Row) 값이 같으면 공격합니다.
      * **대각선 공격**: 두 퀸의 가로 거리(`abs(c1 - c2)`)와 세로 거리(`abs(r1 - r2)`)가 같으면 공격합니다.
      * (수직 공격은 상태 표현 방식에 의해 원천적으로 불가능합니다.)
2.  `result`: 현재 상태(`state`)와 행동(`action`)을 받아, 해당 열의 퀸 위치를 변경한 \*\*새로운 상태(튜플)\*\*를 반환하세요.
      * 행동(Action)은 `(col, new_row)` 형태의 튜플로 정의됩니다.
3.  주어진 `test_state`에 대해 목적 함수 값이 올바르게 계산되는지 확인하세요.



In [1]:
import random
from abc import ABC, abstractmethod
from typing import override

# --- 1. 문제 정의 인터페이스 ---
class Problem[S, A](ABC):
    @abstractmethod
    def initial_state(self) -> S: pass
    @abstractmethod
    def actions(self, state: S) -> list[A]: pass
    @abstractmethod
    def result(self, state: S, action: A) -> S: pass
    @abstractmethod
    def goal_test(self, state: S) -> bool: pass
    @abstractmethod
    def objective_function(self, state: S) -> int: pass

In [2]:
n = 8
random.choices(range(n), k=n)

[3, 6, 5, 1, 1, 1, 1, 7]

In [3]:
random.sample(range(n), n)

[7, 5, 6, 2, 3, 0, 4, 1]

In [4]:
board = list(range(n))
random.shuffle(board)
board

[5, 7, 0, 2, 1, 6, 4, 3]

In [5]:
# --- 2. N-Queens 문제 구체화 ---
class NQueensProblem(Problem[tuple[int, ...], tuple[int, int]]):
    """
    N-Queens 문제를 정의하는 클래스
    State: tuple[int] (각 인덱스는 열, 값은 행을 의미)
    Action: tuple[int, int] (col, new_row) -> 해당 열의 퀸을 새 행으로 이동
    """
    def __init__(self, n: int = 8):
        self.n = n

    @override
    def initial_state(self) -> tuple[int, ...]:
        # 각 열마다 임의의 행에 퀸을 배치
        return random.sample(range(self.n), self.n) # 내가 수정

    @override
    def actions(self, state: tuple[int, ...]) -> list[tuple[int, int]]:
        """
        가능한 모든 이동 생성: 각 열의 퀸을 다른 행으로 이동
        """
        actions = []
        for col in range(self.n):
            current_row = state[col]
            for row in range(self.n):
                if row != current_row:
                    actions.append((col, row))
        return actions

    @override
    def result(self, state: tuple[int, ...], action: tuple[int, int]) -> tuple[int, ...]:
        col, new_row = action

        # Implement your code
        # 튜플은 불변이므로 리스트로 변환 후 수정하고 다시 튜플로 반환하세요.
        next_state = list(state)
        next_state[col] = new_row
        return tuple(next_state)

    @override
    def goal_test(self, state: tuple[int, ...]) -> bool:
        return self.objective_function(state) == 0

    @override
    def objective_function(self, state: tuple[int, ...]) -> int:
        """
        현재 상태의 공격받는 퀸의 쌍(heuristic cost)을 계산 (낮을수록 좋음)
        """
        conflicts = 0
        n = len(state)

        # Implement your code
        for icol, irow in enumerate(state) :
            for jcol in range(icol+1, n) :
                jrow = state[jcol]
                if jrow == irow or abs(icol-jcol) == abs(irow-jrow): conflicts += 1

        return conflicts


In [6]:
# 4-Queens 예제
# (1, 3, 0, 2)는 해답 상태 (conflicts=0)
# (0, 0, 0, 0)은 모든 퀸이 같은 행 (conflicts=6) -> 4C2
problem = NQueensProblem(n=4)

test_state_1 = (0, 0, 0, 0)
print(f"State {test_state_1}: Conflicts = {problem.objective_function(test_state_1)} (Expected: 6)")

test_state_2 = (1, 3, 0, 2)
print(f"State {test_state_2}: Conflicts = {problem.objective_function(test_state_2)} (Expected: 0)")

State (0, 0, 0, 0): Conflicts = 6 (Expected: 6)
State (1, 3, 0, 2): Conflicts = 0 (Expected: 0)


-----

## 실습 문제: Basic Hill Climbing Solver 구현

**설명**:
정의된 `NQueensProblem`을 해결하기 위해 **Basic Hill Climbing(언덕 오르기)** 알고리즘을 구현합니다. 이 알고리즘은 현재 상태에서 이웃한 상태(Neighbors)들을 살펴보고, **가장 좋은(공격 횟수가 가장 적은) 이웃**으로 이동합니다.

Basic Hill Climbing은 단순하지만 \*\*지역 최적해(Local Optima)\*\*에 빠질 위험이 있습니다. 더 나은 이웃이 없다면(즉, 현재 상태보다 공격 횟수가 적은 이웃이 없다면) 탐색을 멈추고 현재 상태를 반환합니다.

**요구사항**:

1.  `solve`:
      * 현재 상태(`current_state`)의 모든 이웃 상태(`neighbors`)를 생성합니다.
      * 모든 이웃 중 목적 함수 값(공격 횟수)이 가장 낮은 \*\*최고의 이웃(`best_neighbor`)\*\*을 찾습니다.
      * **조건**: 만약 `best_neighbor`의 공격 횟수가 현재 상태보다 **크거나 같다면**, 더 이상 개선의 여지가 없다고 판단하고 반복을 종료(break)합니다. (Local Optima 도달).
      * 그렇지 않다면 현재 상태를 `best_neighbor`로 업데이트하고 반복합니다.
2.  실행 결과를 통해 알고리즘이 Global Optimum(공격 횟수 0)을 찾았는지, 아니면 Local Optimum(공격 횟수 \> 0)에 멈췄는지 확인하세요.



In [7]:

# --- 3. Hill Climbing Solver ---
class HillClimbingSolver[S, A]:
    def __init__(self, problem: Problem[S, A]):
        self.problem = problem

    def solve(self) -> tuple[S, int]:
        """
        Basic Hill Climbing 알고리즘 구현
        반환값: (최종 상태, 최종 상태의 conflicts 수)
        """
        current_state = self.problem.initial_state()
        current_score = self.problem.objective_function(current_state)

        steps = 0
        while True:
            steps += 1

            # 1. 현재 상태에서 가능한 모든 행동을 통해 이웃 상태들 생성
            actions = self.problem.actions(current_state)
            neighbors: list[S] = []

            for action in actions:
                neighbors.append(self.problem.result(current_state, action))

            if not neighbors:
                break

            # 2. 이웃 중 가장 점수가 낮은(좋은) 상태 찾기
            # Implement your code
            best_neighbor = None # 수정 필요
            best_score = float('inf') # 수정 필요

            for neighbor in neighbors :
                neighbor_score = self.problem.objective_function(neighbor)

                if neighbor_score >= best_score : continue
                best_score = neighbor_score
                best_neighbor = neighbor




            # 3. 평가 및 이동 결정 (Hill Climbing Logic)
            # 만약 이웃 중 현재보다 확실히 더 좋은 상태가 없다면 멈춤 (>= 사용)
            # Implement your code
            if best_score >= current_score: break

            current_state = best_neighbor
            current_score = best_score

        return current_state, current_score



In [11]:
# --- 4. 실행 및 시각화 ---
def print_nqueens(state: tuple[int, ...]):
    n = len(state)
    print(f"Board (Conflicts: {problem.objective_function(state)})")
    for r in range(n):
        line = ""
        for c in range(n):
            if state[c] == r:
                line += " Q "
            else:
                line += " . "
        print(line)
    print("-" * 20)


# N=8 인 경우
n = 8
problem = NQueensProblem(n=n)
solver = HillClimbingSolver(problem)

print(f"Basic Hill Climbing for {n}-Queens 시작...")
final_state, final_score = solver.solve()

if final_score == 0:
    print("\nGlobal Optimum 도달 (성공)!")
else:
    print(f"\nLocal Optimum에 갇힘 (실패). 남은 공격 쌍: {final_score}")

print_nqueens(final_state)

Basic Hill Climbing for 8-Queens 시작...

Local Optimum에 갇힘 (실패). 남은 공격 쌍: 3
Board (Conflicts: 3)
 .  .  .  .  .  .  Q  . 
 .  .  .  .  Q  .  .  . 
 .  .  Q  .  .  .  .  . 
 .  .  .  .  .  Q  .  . 
 .  .  .  .  .  .  .  Q 
 .  .  .  Q  .  .  .  . 
 .  Q  .  .  .  .  .  . 
 Q  .  .  .  .  .  .  . 
--------------------


## 실습 문제: Random Restart Hill Climbing Solver 구현

**설명**:
기본적인 Hill Climbing 알고리즘은 탐색을 시작하는 초기 상태에 따라 지역 최적해(Local Optima)에 빠질 수 있다는 치명적인 단점이 있습니다. 이를 해결하기 위한 **Random Restart Hill Climbing** 전략은 지역 최적해에 도달하면 무작위 상태에서 탐색을 다시 시작(Restart)하는 방법입니다.

이 문제에서는 앞서 구현한 `HillClimbingSolver`를 내부적으로 사용하여, 글로벌 최적해(Global Optimum, 점수 0)를 찾거나 최대 재시작 횟수에 도달할 때까지 탐색을 반복하는 Solver를 구현합니다.

$$\text{Target Objective}: f(s) = 0$$

**요구사항**:

  - `solve` 메서드 내부에서 `max_restarts` 횟수만큼 반복문을 실행합니다.
  - 각 반복마다 `basic_solver.solve()`를 호출하여 지역 탐색을 수행하고, 반환된 결과(`current_score`)가 지금까지 찾은 `best_score`보다 좋은지 확인하여 업데이트합니다.
  - 만약 탐색된 상태의 점수가 0이라면(Global Optimum), 더 이상의 재시작 없이 즉시 반복을 종료하고 결과를 반환합니다.


In [9]:
import time

class RandomRestartHillClimbingSolver[S, A]:
    def __init__(self, problem: Problem[S, A], max_restarts: int = 100):
        self.problem = problem
        self.max_restarts = max_restarts
        # 내부적으로 사용할 Basic Solver 생성
        self.basic_solver = HillClimbingSolver(problem)

    def solve(self) -> tuple[S, int]:
        """
        Random Restart 전략을 사용하여 문제를 해결합니다.
        Returns:
            (best_state, best_score): 발견된 최고의 상태와 그 점수
        """
        # 최고 기록 초기화
        best_state: S | None = None
        best_score: int | float = float('inf')

        start_time = time.time()

        # 로그 헤더 출력
        print(f"{'Restart #':<10} | {'Final Score':<12} | {'Status'}")
        print("-" * 50)

        for attempt in range(1, self.max_restarts + 1):
            # 1. Basic Hill Climbing 실행

            # Implement your code
            current_state, current_score = self.basic_solver.solve()


            status = "Local Optima"
            is_best_so_far = False

            # 2. 최고 기록 갱신 확인
            if current_score < best_score:
                best_score = current_score
                best_state = current_state
                is_best_so_far = True

            # 상태 메시지 설정 (로깅용)
            if current_score == 0:
                status = "GLOBAL OPTIMUM"
            elif is_best_so_far:
                status = "New Best (Local)"

            print(f"{attempt:<10} | {current_score:<12} | {status}")

            # 3. 목표 달성 시 종료
            if current_score == 0:
                break

        elapsed_time = time.time() - start_time
        print("-" * 50)
        print(f"Summary:")
        print(f"   - Total Restarts : {attempt}")
        print(f"   - Execution Time : {elapsed_time:.4f} sec")
        print(f"   - Best Score     : {best_score}")

        return best_state, best_score

In [10]:
def print_nqueens(state: tuple[int, ...]):
    n = len(state)
    print(f"Board (Conflicts: {problem.objective_function(state)})")
    for r in range(n):
        line = ""
        for c in range(n):
            if state[c] == r:
                line += " Q "
            else:
                line += " . "
        print(line)
    print("-" * 20)

# 1. 문제 설정 (N=15)
n_queens = 15
print(f"{n_queens}-Queens 문제 해결을 시작합니다 (Random Restart Hill Climbing)...")
print("=" * 60)

# 2. 문제 인스턴스 및 솔버 생성
# (NQueensProblem, HillClimbingSolver 클래스는 이미 정의되어 있어야 합니다)
problem = NQueensProblem(n=n_queens)

# 최대 50번까지 재시작
solver = RandomRestartHillClimbingSolver(problem, max_restarts=50)

# 3. 알고리즘 실행
best_state, best_score = solver.solve()

# 4. 결과 시각화 및 출력
print("\n" + "="*40)
print(f"최종 결과 (Best Score: {best_score})")
print("="*40)

if best_score == 0:
    print(f"{n_queens}-Queens 최적해를 찾았습니다!\n")
    print("   [Queens Placement Visualization]")

    print_nqueens(best_state)

    print(f"\nFound State Tuple: {best_state}")
else:
    print("아쉽게도 최적해(Score 0)를 찾지 못했습니다.")
    print(f"발견한 가장 좋은 상태: {best_state}")

15-Queens 문제 해결을 시작합니다 (Random Restart Hill Climbing)...
Restart #  | Final Score  | Status
--------------------------------------------------
1          | 3            | New Best (Local)
2          | 3            | Local Optima
3          | 3            | Local Optima
4          | 3            | Local Optima
5          | 3            | Local Optima
6          | 2            | New Best (Local)
7          | 2            | Local Optima
8          | 1            | New Best (Local)
9          | 1            | Local Optima
10         | 4            | Local Optima
11         | 2            | Local Optima
12         | 3            | Local Optima
13         | 2            | Local Optima
14         | 3            | Local Optima
15         | 3            | Local Optima
16         | 3            | Local Optima
17         | 2            | Local Optima
18         | 4            | Local Optima
19         | 2            | Local Optima
20         | 5            | Local Optima
21         | 1           